### Today we are going to track experiments of our model using TensorBoard

In [13]:
import torch
import torchvision

from torch import nn
from torchvision import transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

from torchvision import datasets
import matplotlib.pyplot as plt
from torchinfo import summary

import requests
import os
import zipfile


In [40]:
# Create Data Loader

train = os.path.join('Dataset_Pizza_Steak_Sushi','train')
test = os.path.join('Dataset_Pizza_Steak_Sushi','test')


# transform data to Tensor

data_transform = transforms.Compose([
  transforms.Resize((64, 64)),
  transforms.ToTensor()
])

train_data = datasets.ImageFolder(train,transform=data_transform)
test_data = datasets.ImageFolder(test,transform=data_transform)

class_names = train_data.classes


train_dataloader = DataLoader(train_data,
    batch_size=16, 
    shuffle=True )

test_dataloader = DataLoader(test_data,
    batch_size=16, 
    shuffle=True )


In [41]:
train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x2d8f1c9f4f0>,
 ['pizza', 'steak', 'sushi'])

### We will use a pre-trained model of EfficientNet for our problem

In [9]:
# Download the pretrained weights for EfficientNet_B0
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT 

# Setup the model with the pretrained weights and send it to the target device
model = torchvision.models.efficientnet_b0(weights=weights)

model

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to C:\Users\LT/.cache\torch\hub\checkpoints\efficientnet_b0_rwightman-3dd342df.pth


  0%|          | 0.00/20.5M [00:00<?, ?B/s]

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [30]:
# Lets view the summary of model

summary(model=model,
         input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape"
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 1000]           --                   True
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   True
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   True
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   864                  True
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   64                   True
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 16, 112

# Train the model and track the results

In [35]:
from torch.utils.tensorboard import SummaryWriter

#Create a summary writer
writer = SummaryWriter()

### Make functions for training and testing

In [52]:

def train_step(
               model:torch.nn.Module,
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer
         ):
    
    
    model.train()
    
    train_loss, train_acc = 0, 0
    
    for batch, (X,y) in enumerate(dataloader):
        
        # Forward Pass
        y_pred = model(X)
        
        loss = loss_fn(y_pred, y)
        train_loss += loss.item() 

       # Optimizer zero grad
        optimizer.zero_grad()

      # Loss backward
        loss.backward()

      # Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)
    
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

        

In [53]:
def test_step(
              model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module
              ):
    
    model.eval()
    test_loss, test_acc = 0, 0
    with torch.inference_mode():
        for batch, (X,y) in enumerate(dataloader):
        #   Forward Pass     
            test_pred = model(X)
            
            # Calculate Loss
            loss = loss_fn(test_pred,y)
            test_loss += loss.item()
            
            #Calculae Accuracy
            test_pred_labels =  torch.argmax(torch.softmax(test_pred,dim=1),dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))
    
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    
    return test_loss, test_acc

In [54]:
# train function

def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int):
    
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }

    # Loop through training and testing steps for a number of epochs
    for epoch in range(epochs):
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer)
        
        test_loss, test_acc = test_step(model=model,
                                        dataloader=test_dataloader,
                                        loss_fn=loss_fn)
        
        
        # Print training
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
        
        # Add loss results to SummaryWriter
        writer.add_scalars(main_tag="Loss", 
                           tag_scalar_dict={"train_loss": train_loss,
                                            "test_loss": test_loss},
                           global_step=epoch)

        # Add accuracy results to SummaryWriter
        writer.add_scalars(main_tag="Accuracy", 
                           tag_scalar_dict={"train_acc": train_acc,
                                            "test_acc": test_acc}, 
                           global_step=epoch)
        
         # Track the PyTorch model architecture
        writer.add_graph(model=model,input_to_model=torch.randn(32, 3, 224, 224))
                         
    #close writer
    writer.close()
                         
    return results
    

In [55]:
# Run the model

# Define loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

results = train(model=model,
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                optimizer=optimizer,
                loss_fn=loss_fn,
                epochs=5)

Epoch: 1 | train_loss: 6.6041 | train_acc: 0.1000 | test_loss: 4.5876 | test_acc: 0.3545
Epoch: 2 | train_loss: 2.0470 | train_acc: 0.7208 | test_loss: 2.8855 | test_acc: 0.6466
Epoch: 3 | train_loss: 1.0646 | train_acc: 0.7958 | test_loss: 2.0655 | test_acc: 0.7261
Epoch: 4 | train_loss: 0.6735 | train_acc: 0.8750 | test_loss: 1.5582 | test_acc: 0.8011
Epoch: 5 | train_loss: 0.4193 | train_acc: 0.9417 | test_loss: 1.7419 | test_acc: 0.7591


In [56]:
#Results

results

{'train_loss': [6.60411057472229,
  2.0469865083694456,
  1.0646150449911753,
  0.6735406046112379,
  0.41933495476841925],
 'train_acc': [0.1,
  0.7208333333333333,
  0.7958333333333333,
  0.875,
  0.9416666666666667],
 'test_loss': [4.587566900253296,
  2.885534739494324,
  2.0654690980911257,
  1.5582238793373109,
  1.7418508172035216],
 'test_acc': [0.35454545454545455,
  0.6465909090909091,
  0.7261363636363637,
  0.8011363636363636,
  0.759090909090909]}

## View Summary Writer Results in tensorboard

In [58]:
# Example code to run in Jupyter or Google Colab Notebook (uncomment to try it out)
%load_ext tensorboard
%tensorboard --logdir runs